In [ ]:
import os
current_directory = os.getcwd()
publish_local = current_directory.replace("monte_carlo", "libraries\Vts.dll")

In [ ]:
pip install pythonnet plotly numpy

In [ ]:
from pythonnet import set_runtime
set_runtime("coreclr")
import clr
clr.AddReference(publish_local) # Copy the VTS dlls into the libraries folders
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from Vts import *
from Vts.Common import *
from Vts.Extensions import *
from Vts.Modeling.Optimizers import *
from Vts.Modeling.ForwardSolvers import *
from Vts.SpectralMapping import *
from Vts.Factories import *
from Vts.MonteCarlo import *
from Vts.MonteCarlo.Sources import *
from Vts.MonteCarlo.Tissues import *
from Vts.MonteCarlo.Detectors import *
from Vts.MonteCarlo.Factories import *
from Vts.MonteCarlo.PhotonData import *
from Vts.MonteCarlo.PostProcessing import *
from System import Array

In [ ]:
# create a SimulationInput object to define the simulation
detectorRange = DoubleRange(start=0, stop=40, number=201)
detectorInput = ROfRhoDetectorInput()
detectorInput.Rho = detectorRange
detectorInput.Name = "ROfRho"
detectors = Array.CreateInstance(IDetectorInput,1)
detectors[0] = detectorInput

simulationInput = SimulationInput()
simulationInput.N=1000
simulationInput.DetectorInputs= detectors

# create the simulation
simulation = MonteCarloSimulation(simulationInput)

# run the simulation
simulationOutput = simulation.Run()

# plot the results using Plotly
detectorResults = Array.CreateInstance(ROfRhoDetector,1)
detectorResults[0] = simulationOutput.ResultsDictionary["ROfRho"]
logReflectance = [r for r in detectorResults[0].Mean]
detectorMidpoints = [mp for mp in detectorRange.AsEnumerable()]

xLabel = "ρ [mm]"
yLabel = "log(R(ρ)) [mm-2]"

chart = go.Figure()
chart.add_trace(go.Scatter(x=detectorMidpoints, y=logReflectance, mode='lines+markers'))
chart.update_layout( title="log(R(ρ)) [mm-2]", xaxis_title=xLabel, yaxis_title=yLabel)
chart.update_yaxes(type="log")
chart.show()
